In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('imdb_cleaned.csv')

In [3]:
df.head()

,Unnamed: 0,review_text,sentiment_score,review_stopwords_noise_remove
0,0,one of the other reviewers has mentioned that ...,positive,one reviewer mention watch 1 episode hook righ...
1,1,a wonderful little production. <br /><br />the...,positive,wonderful little production film technique una...
2,2,i thought this was a wonderful way to spend ti...,positive,think wonderful way spend time hot summer week...
3,3,basically there's a family where a little boy ...,negative,basically there family little boy jake think t...
4,4,"petter mattei's ""love in the time of money"" is...",positive,petter love time money visually stunning film ...


In [4]:
df.drop(['Unnamed: 0','review_text','sentiment_score'],axis=1,inplace=True)

In [5]:
df.head()

,review_stopwords_noise_remove
0,one reviewer mention watch 1 episode hook righ...
1,wonderful little production film technique una...
2,think wonderful way spend time hot summer week...
3,basically there family little boy jake think t...
4,petter love time money visually stunning film ...


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

In [7]:
vectorizer=TfidfVectorizer()
x=vectorizer.fit_transform(df['review_stopwords_noise_remove'].values).toarray().astype(int)

In [9]:
x.shape

(50000, 39303)

In [10]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=TFBertModel.from_pretrained('bert-base-uncased',trainable=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [11]:
batch_size=100
embeddings=[]
for i in range (0,len(df['review_stopwords_noise_remove']),batch_size):
    batch_data=tokenizer.batch_encode_plus(
               df['review_stopwords_noise_remove'][i : i+batch_size],
               max_length=128,
               padding='max_length',
               truncation=True,
               return_tensors='tf')
    output=model(**batch_data)
    pooled_output=output.pooler_output
    embeddings.append(pooled_output)

In [12]:
x_1=tf.concat(embeddings,axis=0).numpy()

In [13]:
x_1.shape

(50000, 768)

In [14]:
x_final=np.hstack((x,x_1))

In [15]:
x_final.shape

(50000, 40071)

In [16]:
del x,x_1

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
df['similarity_score']=0

In [19]:
similarity_matrix = cosine_similarity(x_final)

In [20]:
similarity_matrix[0][3]

0.9618298146255453